In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import os, re
import datetime
import pandas as pd
from IPython.display import display
import math
import numpy as np
import platform

In [2]:
from base64 import b64decode
from io import BytesIO

from IPython import get_ipython
from IPython.core.magic import register_cell_magic
import PIL


@register_cell_magic
def capture_png(line, cell):
    get_ipython().run_cell_magic(
        'capture',
        ' --no-stderr --no-stdout result',
        cell
    )
    out_paths = line.strip().split(' ')
    for output in result.outputs:
        data = output.data
        if 'image/png' in data:
            path = out_paths.pop(0)
            if not path:
                raise ValueError('Too few paths given!')
            png_bytes = data['image/png']
            if isinstance(png_bytes, str):
                png_bytes = b64decode(png_bytes)
            assert isinstance(png_bytes, bytes)
            bytes_io = BytesIO(png_bytes)
            image = PIL.Image.open(bytes_io)
            image.save(path, 'png')

In [3]:
#Read in Precip data files

base_dir='/media/Work/SandBox/starbuck' if re.match(r'^.*(stewmanji).*$',platform.node(), flags=re.I) else '/Users/meghan/Lemhi_Main'
root_dir =   os.path.join(base_dir, 'Lemhi/CONUS404/CONUS404_Data/C404_area_avgs')
LRidge = nc.Dataset(os.path.join(root_dir, 'lemhiridge.nc'),'r')
BLower =  nc.Dataset(os.path.join(root_dir, 'beaverheadlower.nc'),'r')
BUpper= nc.Dataset(os.path.join(root_dir, 'beaverheadupper.nc'),'r')
LELower= nc.Dataset(os.path.join(root_dir, 'lemhieastlower.nc'),'r')
LEUpper= nc.Dataset(os.path.join(root_dir, 'lemhieastupper.nc'),'r')
PNorth= nc.Dataset(os.path.join(root_dir, 'pahsimeroinorth.nc'),'r')
JLMeadow= nc.Dataset(os.path.join(root_dir, 'jlmeadow.nc'),'r')
Schwartz= nc.Dataset(os.path.join(root_dir, 'schwartz.nc'),'r')
Gunsight= nc.Dataset(os.path.join(root_dir, 'gunsight.nc'),'r')
Moonshine= nc.Dataset(os.path.join(root_dir, 'moonshine.nc'),'r')
Lemhi_Lower_Sfc =nc.Dataset(os.path.join(root_dir, 'lemhinorthsurface.nc'),'r')
Lemhi_Upper_Sfc = nc.Dataset(os.path.join(root_dir, 'lemhisouthsurface.nc'),'r')
Ezra_Creek = nc.Dataset(os.path.join(root_dir, 'ezracreek.nc'),'r')
Challis = nc.Dataset(os.path.join(root_dir, 'challisairport.nc'),'r')
Pahs_Sfc = nc.Dataset(os.path.join(root_dir, 'pahsimeroisurface.nc'),'r')



In [4]:
#Froude Number

froude_base_dir='/media/Work/SandBox/starbuck/Lemhi/CONUSI/CI_Data/CI_froude' if re.match(r'^.*(stewmanji).*$',platform.node(), flags=re.I) else '/Users/meghan/Lemhi_Main/Lemhi/CONUS404/CONUS404_Data/froude'
Fr_A = nc.Dataset(os.path.join(froude_base_dir, 'generator_a.nc'),'r')
Fr_B = nc.Dataset(os.path.join(froude_base_dir, 'generator_b.nc'),'r')
Fr_D = nc.Dataset(os.path.join(froude_base_dir, 'generator_d.nc'),'r')
Fr_G = nc.Dataset(os.path.join(froude_base_dir, 'generator_g.nc'),'r')
Fr_H = nc.Dataset(os.path.join(froude_base_dir, 'generator_h.nc'),'r')
Fr_I = nc.Dataset(os.path.join(froude_base_dir, 'generator_i.nc'),'r')

#Generator A
BH_A = Fr_A.variables['FrBH'][:]
LH_A = Fr_A.variables['FrLH'][:]
LR_A = Fr_A.variables['FrLR'][:]
Fr_A_Time = Fr_A.variables['Time'][:]

#Generator B
BH_B = Fr_B.variables['FrBH'][:]
LH_B = Fr_B.variables['FrLH'][:]
LR_B = Fr_B.variables['FrLR'][:]
Fr_B_Time = Fr_B.variables['Time'][:]

#Generator D
BH_D = Fr_D.variables['FrBH'][:]
LH_D = Fr_D.variables['FrLH'][:]
LR_D = Fr_D.variables['FrLR'][:]
Fr_D_Time = Fr_D.variables['Time'][:]

#Generator G
BH_G = Fr_G.variables['FrBH'][:]
LH_G = Fr_G.variables['FrLH'][:]
LR_G = Fr_G.variables['FrLR'][:]
Fr_G_Time = Fr_G.variables['Time'][:]

#Generator H
BH_H = Fr_H.variables['FrBH'][:]
LH_H = Fr_H.variables['FrLH'][:]
LR_H = Fr_H.variables['FrLR'][:]
Fr_H_Time = Fr_H.variables['Time'][:]

#Generator I
BH_I = Fr_I.variables['FrBH'][:]
LH_I = Fr_I.variables['FrLH'][:]
LR_I = Fr_I.variables['FrLR'][:]
Fr_I_Time = Fr_I.variables['Time'][:]

gen_a = nc.Dataset(os.path.join(root_dir, 'generator_a.nc'),'r')
gen_b = nc.Dataset(os.path.join(root_dir, 'generator_b.nc'),'r')
gen_d = nc.Dataset(os.path.join(root_dir, 'generator_d.nc'),'r')
gen_g = nc.Dataset(os.path.join(root_dir, 'generator_g.nc'),'r')
gen_h = nc.Dataset(os.path.join(root_dir, 'generator_h.nc'),'r')
gen_i = nc.Dataset(os.path.join(root_dir, 'generator_i.nc'),'r')

#Generator A
A_Time =gen_a.variables['Time'][:] #Hours since 1/1/1901 00:00:00
A_Temp =gen_a.variables['Tc_700MB'][:] #Temperature degrees C
A_Precip =gen_a.variables['PREC_ACC_NC'][:] #Precip accumulation
A_Air_LWC =gen_a.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
A_Air_Temp =gen_a.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
A_Ground_LWC =gen_a.variables['GS_LWC'][:] #Ground LWC Seedable
A_Ground_Temp =gen_a.variables['GS_Tc'][:] #Precip accumulation degrees C
A_U =gen_a.variables['U_700MB'][:] #U Wind 700MB
A_V =gen_a.variables['V_700MB'][:] #V Wind 700MB
A_windDir = np.arctan2(A_U,A_V)*180/math.pi+180

#Generator B
B_Time =gen_b.variables['Time'][:] #Hours since 1/1/1901 00:00:00
B_Temp =gen_b.variables['Tc_700MB'][:] #Temperature degrees C
B_Precip =gen_b.variables['PREC_ACC_NC'][:] #Precip accumulation
B_Air_LWC =gen_b.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
B_Air_Temp =gen_b.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
B_Ground_LWC =gen_b.variables['GS_LWC'][:] #Ground LWC Seedable
B_Ground_Temp =gen_b.variables['GS_Tc'][:] #Precip accumulation degrees C
B_U =gen_b.variables['U_700MB'][:] #U Wind 700MB
B_V =gen_b.variables['V_700MB'][:] #V Wind 700MB
B_windDir = np.arctan2(B_U,B_V)*180/math.pi+180

#Generator D
D_Time =gen_d.variables['Time'][:] #Hours since 1/1/1901 00:00:00
D_Temp =gen_d.variables['Tc_700MB'][:] #Temperature degrees C
D_Precip =gen_d.variables['PREC_ACC_NC'][:] #Precip accumulation
D_Air_LWC =gen_d.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
D_Air_Temp =gen_d.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
D_Ground_LWC =gen_d.variables['GS_LWC'][:] #Ground LWC Seedable
D_Ground_Temp =gen_d.variables['GS_Tc'][:] #Precip accumulation degrees C
D_U =gen_d.variables['U_700MB'][:] #U Wind 700MB
D_V =gen_d.variables['V_700MB'][:] #V Wind 700MB
D_windDir = np.arctan2(D_U,D_V)*180/math.pi+180

#Generator G
G_Time =gen_g.variables['Time'][:] #Hours since 1/1/1901 00:00:00
G_Temp =gen_g.variables['Tc_700MB'][:] #Temperature degrees C
G_Precip =gen_g.variables['PREC_ACC_NC'][:] #Precip accumulation
G_Air_LWC =gen_g.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
G_Air_Temp =gen_g.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
G_Ground_LWC =gen_g.variables['GS_LWC'][:] #Ground LWC Seedable
G_Ground_Temp =gen_g.variables['GS_Tc'][:] #Precip accumulation degrees C
G_U =gen_g.variables['U_700MB'][:] #U Wind 700MB
G_V =gen_g.variables['V_700MB'][:] #V Wind 700MB
G_windDir = np.arctan2(G_U,G_V)*180/math.pi+180

#Generator H
H_Time =gen_h.variables['Time'][:] #Hours since 1/1/1901 00:00:00
H_Temp =gen_h.variables['Tc_700MB'][:] #Temperature degrees C
H_Precip =gen_h.variables['PREC_ACC_NC'][:] #Precip accumulation
H_Air_LWC =gen_h.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
H_Air_Temp =gen_h.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
H_Ground_LWC =gen_h.variables['GS_LWC'][:] #Ground LWC Seedable
H_Ground_Temp =gen_h.variables['GS_Tc'][:] #Precip accumulation degrees C
H_U =gen_h.variables['U_700MB'][:] #U Wind 700MB
H_V =gen_h.variables['V_700MB'][:] #V Wind 700MB
H_windDir = np.arctan2(H_U,H_V)*180/math.pi+180

#Generator I
I_Time =gen_i.variables['Time'][:] #Hours since 1/1/1901 00:00:00
I_Temp =gen_i.variables['Tc_700MB'][:] #Temperature degrees C
I_Precip =gen_i.variables['PREC_ACC_NC'][:] #Precip accumulation
I_Air_LWC =gen_i.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
I_Air_Temp =gen_i.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
I_Ground_LWC =gen_i.variables['GS_LWC'][:] #Ground LWC Seedable
I_Ground_Temp =gen_i.variables['GS_Tc'][:] #Precip accumulation degrees C
I_U =gen_i.variables['U_700MB'][:] #U Wind 700MB
I_V =gen_i.variables['V_700MB'][:] #V Wind 700MB
I_windDir = np.arctan2(I_U,I_V)*180/math.pi+180


In [5]:

LRidge_Time = LRidge.variables['Time'][:]
LRidge_AS_LWC = LRidge.variables['AS_LWC'][:]
LRidge_GS_LWC = LRidge.variables['GS_LWC'][:]
LRidge_Precip = LRidge.variables['PREC_ACC_NC'][:]
LRidge_Temp = LRidge.variables['AS_Tc'][:]
LRidge_GS_Temp = LRidge.variables['GS_Tc'][:]

JLMeadow_Time = JLMeadow.variables['Time'][:]
JLMeadow_AS_LWC = JLMeadow.variables['AS_LWC'][:]
JLMeadow_GS_LWC = JLMeadow.variables['GS_LWC'][:]
JLMeadow_Precip = JLMeadow.variables['PREC_ACC_NC'][:]
JLMeadow_Temp = JLMeadow.variables['AS_Tc'][:]
JLMeadow_GS_Temp = JLMeadow.variables['GS_Tc'][:]

Schwartz_Time = Schwartz.variables['Time'][:]
Schwartz_AS_LWC = Schwartz.variables['AS_LWC'][:]
Schwartz_GS_LWC = Schwartz.variables['GS_LWC'][:]
Schwartz_Precip = Schwartz.variables['PREC_ACC_NC'][:]
Schwartz_Temp = Schwartz.variables['AS_Tc'][:]
Schwartz_GS_Temp = Schwartz.variables['GS_Tc'][:]

Gunsight_Time = Gunsight.variables['Time'][:]
Gunsight_AS_LWC = Gunsight.variables['AS_LWC'][:]
Gunsight_GS_LWC = Gunsight.variables['GS_LWC'][:]
Gunsight_Precip = Gunsight.variables['PREC_ACC_NC'][:]
Gunsight_GS_Temp = Gunsight.variables['GS_Tc'][:]

Moonshine_Time = Moonshine.variables['Time'][:]
Moonshine_AS_LWC = Moonshine.variables['AS_LWC'][:]
Moonshine_GS_LWC = Moonshine.variables['GS_LWC'][:]
Moonshine_Precip = Moonshine.variables['PREC_ACC_NC'][:]
Moonshine_Temp = Moonshine.variables['AS_Tc'][:]
Moonshine_GS_Temp = Moonshine.variables['GS_Tc'][:]

BLower_Time = BLower.variables['Time'][:]
BLower_AS_LWC = BLower.variables['AS_LWC'][:]
BLower_GS_LWC = BLower.variables['GS_LWC'][:]
BLower_Precip = BLower.variables['PREC_ACC_NC'][:]
BLower_Temp = BLower.variables['AS_Tc'][:]
BLower_GS_Temp = BLower.variables['GS_Tc'][:]

BUpper_Time = BUpper.variables['Time'][:]
BUpper_AS_LWC = BUpper.variables['AS_LWC'][:]
BUpper_GS_LWC = BUpper.variables['GS_LWC'][:]
BUpper_Precip = BUpper.variables['PREC_ACC_NC'][:]
BUpper_Temp = BUpper.variables['AS_Tc'][:]
BUpper_GS_Temp = BUpper.variables['GS_Tc'][:]

LEUpper_Time = LEUpper.variables['Time'][:]
LEUpper_AS_LWC = LEUpper.variables['AS_LWC'][:]
LEUpper_GS_LWC = LEUpper.variables['GS_LWC'][:]
LEUpper_Precip = LEUpper.variables['PREC_ACC_NC'][:]
LEUpper_Temp = LEUpper.variables['AS_Tc'][:]
LEUpper_GS_Temp = LEUpper.variables['GS_Tc'][:]

LELower_Time = LELower.variables['Time'][:]
LELower_AS_LWC = LELower.variables['AS_LWC'][:]
LELower_GS_LWC = LELower.variables['GS_LWC'][:]
LELower_Precip = LELower.variables['PREC_ACC_NC'][:]
LELower_Temp = LELower.variables['AS_Tc'][:]
LELower_GS_Temp = LELower.variables['GS_Tc'][:]

PNorth_Time = PNorth.variables['Time'][:]
PNorth_AS_LWC = PNorth.variables['AS_LWC'][:]
PNorth_GS_LWC = PNorth.variables['GS_LWC'][:]
PNorth_Precip = PNorth.variables['PREC_ACC_NC'][:]
PNorth_Temp = PNorth.variables['AS_Tc'][:]
PNorth_GS_Temp = PNorth.variables['GS_Tc'][:]



In [6]:

#Lemhi Lower Surface
LLowerSfc_Time =Lemhi_Lower_Sfc.variables['Time'][:] #Hours since 1/1/1901 00:00:00
LLowerSfc_Temp =Lemhi_Lower_Sfc.variables['Tc_700MB'][:] #Temperature degrees C
LLowerSfc_Precip =Lemhi_Lower_Sfc.variables['PREC_ACC_NC'][:] #Precip accumulation
LLowerSfc_Air_LWC =Lemhi_Lower_Sfc.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
LLowerSfc_Air_Temp =Lemhi_Lower_Sfc.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
LLowerSfc_Ground_LWC =Lemhi_Lower_Sfc.variables['GS_LWC'][:] #Ground LWC Seedable
LLowerSfc_Ground_Temp =Lemhi_Lower_Sfc.variables['GS_Tc'][:] #Precip accumulation degrees C
LLowerSfc_U =Lemhi_Lower_Sfc.variables['U_700MB'][:] #U Wind 700MB
LLowerSfc_V =Lemhi_Lower_Sfc.variables['V_700MB'][:] #V Wind 700MB
LLowerSfc_windDir = np.arctan2(LLowerSfc_U,LLowerSfc_V)*180/math.pi+180

#Lemhi Upper Surface
LUpperSfc_Time =Lemhi_Upper_Sfc.variables['Time'][:] #Hours since 1/1/1901 00:00:00
LUpperSfc_Temp =Lemhi_Upper_Sfc.variables['Tc_700MB'][:] #Temperature degrees C
LUpperSfc_Precip =Lemhi_Upper_Sfc.variables['PREC_ACC_NC'][:] #Precip accumulation
LUpperSfc_Air_LWC =Lemhi_Upper_Sfc.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
LUpperSfc_Air_Temp =Lemhi_Upper_Sfc.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
LUpperSfc_Ground_LWC =Lemhi_Upper_Sfc.variables['GS_LWC'][:] #Ground LWC Seedable
LUpperSfc_Ground_Temp =Lemhi_Upper_Sfc.variables['GS_Tc'][:] #Precip accumulation degrees C
LUpperSfc_U =Lemhi_Upper_Sfc.variables['U_700MB'][:] #U Wind 700MB
LUpperSfc_V =Lemhi_Upper_Sfc.variables['V_700MB'][:] #V Wind 700MB
LUpperSfc_windDir = np.arctan2(LUpperSfc_U,LUpperSfc_V)*180/math.pi+180

#Ezra Creek
Ezra_Time =Ezra_Creek.variables['Time'][:] #Hours since 1/1/1901 00:00:00
Ezra_Temp =Ezra_Creek.variables['Tc_700MB'][:] #Temperature degrees C
Ezra_Precip =Ezra_Creek.variables['PREC_ACC_NC'][:] #Precip accumulation
Ezra_Air_LWC =Ezra_Creek.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
Ezra_Air_Temp =Ezra_Creek.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
Ezra_Ground_LWC =Ezra_Creek.variables['GS_LWC'][:] #Ground LWC Seedable
Ezra_Ground_Temp =Ezra_Creek.variables['GS_Tc'][:] #Precip accumulation degrees C
Ezra_U =Ezra_Creek.variables['U_700MB'][:] #U Wind 700MB
Ezra_V =Ezra_Creek.variables['V_700MB'][:] #V Wind 700MB
Ezra_windDir = np.arctan2(Ezra_U,Ezra_V)*180/math.pi+180

#Pahsimeroi Surface 
PahsSfc_Time =Pahs_Sfc.variables['Time'][:] #Hours since 1/1/1901 00:00:00
PahsSfc_Temp =Pahs_Sfc.variables['Tc_700MB'][:] #Temperature degrees C
PahsSfc_Precip =Pahs_Sfc.variables['PREC_ACC_NC'][:] #Precip accumulation
PahsSfc_Air_LWC =Pahs_Sfc.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
PahsSfc_Air_Temp =Pahs_Sfc.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
PahsSfc_Ground_LWC =Pahs_Sfc.variables['GS_LWC'][:] #Ground LWC Seedable
PahsSfc_Ground_Temp =Pahs_Sfc.variables['GS_Tc'][:] #Precip accumulation degrees C
PahsSfc_U =Pahs_Sfc.variables['U_700MB'][:] #U Wind 700MB
PahsSfc_V =Pahs_Sfc.variables['V_700MB'][:] #V Wind 700MB
PahsSfc_windDir = np.arctan2(PahsSfc_U,PahsSfc_V)*180/math.pi+180

#Challis Airport
Challis_Time =Challis.variables['Time'][:] #Hours since 1/1/1901 00:00:00
Challis_Temp =Challis.variables['Tc_700MB'][:] #Temperature degrees C
Challis_Precip =Challis.variables['PREC_ACC_NC'][:] #Precip accumulation
Challis_Air_LWC =Challis.variables['AS_LWC'][:] #Airborne Layer LWC Seedable
Challis_Air_Temp =Challis.variables['AS_Tc'][:] #Airborne Layer Temp degrees C
Challis_Ground_LWC =Challis.variables['GS_LWC'][:] #Ground LWC Seedable
Challis_Ground_Temp =Challis.variables['GS_Tc'][:] #Precip accumulation degrees C
Challis_U =Challis.variables['U_700MB'][:] #U Wind 700MB
Challis_V =Challis.variables['V_700MB'][:] #V Wind 700MB
Challis_windDir = np.arctan2(Challis_U,Challis_V)*180/math.pi+180

## Lemhi Ridge	Beaverhead Lower Lemhi Lower Surface	225-315	G	Fr_BH

| Precip      | Criteria         | Surface     | Wind    | Generator  | Froude Var |
| ----------- | ---------------- | ----------- | ------- | ---------- | ---------- |
| Lemhi Ridge | Beaverhead Lower | Lemhi Lower | 225-315 | G          | Fr_BH      |



In [8]:
#Lemhi Ridge	Beaverhead Lower Lemhi Lower Surface	225-315	G	Fr_BH
precip_df = pd.DataFrame({'time':LRidge_Time, 'precip' :LRidge_Precip})
criteria_df = pd.DataFrame({'time':BLower_Time, 'temp':BLower_GS_Temp,
                             'lwc':BLower_GS_LWC})
wind_df = pd.DataFrame({'time':G_Time, 'wind':G_windDir})  
froude_df = pd.DataFrame({'time':Fr_G_Time, 'Fr':BH_G})
precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)
wind_df.set_index('time', drop=True, inplace=True)
froude_df.set_index('time', drop=True, inplace=True)


_odf = criteria_df.merge(precip_df, on = 'time').merge(wind_df,on='time').merge(froude_df,on='time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*
_wind_lim = (225, 315)
def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit_df = _crit_df[_crit_df['temp'] > -18]
    _crit_df = _crit_df[_crit_df['lwc']  > 0.01]
    _wind_df = _crit_df[_crit_df['wind'] > 225]
    _wind_df = _wind_df[_wind_df['wind'] < 315]
    # _wind_df = _crit_df[_crit_df['wind']>_wind_lim[0]]
    # _wind_df = _wind_df[_wind_df['wind'] <_wind_lim[1]]
    _froude_df = _wind_df[_wind_df['Fr'] > 0.5]
    
#     _pyear_df = _odf[_odf.index.year == (year-1)]
#     _py_dfs=[_pyear_df[_pyear_df.index.month == mm] for mm in [11,12]]
#     _cy_dfs=[_year_df[_year_df.index.month == mm] for mm in [1,2,3,4]]
#     _wy_precip_df = pd.concat(_py_dfs+_cy_dfs)
    
#     _wy_crit_df = _wy_precip_df[_wy_precip_df['temp'] < -6]
#     _wy_crit_df = _wy_crit_df[_wy_crit_df['temp'] > -18]
#     _wy_crit_df = _wy_crit_df[_wy_crit_df['lwc'] > 0.01]
#     _wy_wind_df = _wy_crit_df[_wy_crit_df['wind']>225]
#     _wy_wind_df = _wy_wind_df[_wy_wind_df['wind'] <315]
#     _wy_froude_df = _wy_wind_df[_wy_wind_df['Fr'] > 0.5]
    
    return {
        'year':year, 
        'Calendar Year Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Calendar Year Seedable Precip':(_froude_df.precip.sum()),
        # 'Water Year Total Precip':(_wy_precip_df.precip.sum()),
        # 'Water Year Seedable Precip':(_wy_froude_df.precip.sum()),
        
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Lemhi Lower Surface'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Calendar Year Total Precip,Calendar Year Seedable Precip
count,1.0,1.0
mean,0.0,0.0
std,NaN,NaN
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


In [ ]:
print(df.describe().to_markdown())

## JLMeadow Precip with Beaverhead Upper Criteria

| Precip   | Criteria         | Wind    | Generator  | Froude Var |
| -------- | ---------------- | ------- | ---------- | ---------- |
| JLMeadow | Beaverhead Upper | 225-315 | H          | Fr_BH      |

In [44]:
#JLMeadow Precip with Beaverhead Upper Criteria
precip_df = pd.DataFrame({'time':JLMeadow_Time, 'precip' :JLMeadow_Precip})
criteria_df = pd.DataFrame({'time':BUpper_Time, 'temp':BUpper_GS_Temp,
                             'lwc':BUpper_GS_LWC})
wind_df = pd.DataFrame({'time':H_Time, 'wind':H_windDir})  
froude_df = pd.DataFrame({'time':Fr_H_Time, 'Fr':BH_H})
precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)
wind_df.set_index('time', drop=True, inplace=True)
froude_df.set_index('time', drop=True, inplace=True)

_wind_lim = (225, 315)

_odf = criteria_df.merge(precip_df, on = 'time').merge(wind_df,on='time').merge(froude_df,on='time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

df = collect_seasons()

df['Region']='Lemhi Upper Surface'
df['Precip']='JL Meadow'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,329.869164,44.961615
std,69.097247,15.652827
min,182.655200,16.026058
25%,287.813522,31.265579
50%,326.767671,44.450132
75%,365.688702,54.509914
max,475.874546,79.136784


In [ ]:
print(df.describe().to_markdown())

## Schwartz Precip with Lemhi East Lower Criteria

| Precip   | Criteria         | Wind    | Generator  | Froude Var |
| -------- | ---------------- | ------- | ---------- | ---------- |
| Schwartz | Lemhi East Lower | 225-340 | A          | Fr_LH      |



In [47]:
#Schwartz Precip with Lemhi East Lower Criteria
precip_df = pd.DataFrame({'time':Schwartz_Time, 'precip' :Schwartz_Precip})
criteria_df = pd.DataFrame({'time':LELower_Time, 'temp':LELower_GS_Temp,
                             'lwc':LELower_GS_LWC})
wind_df = pd.DataFrame({'time':A_Time, 'wind':A_windDir})  
froude_df = pd.DataFrame({'time':Fr_A_Time, 'Fr':LH_A})
precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)
wind_df.set_index('time', drop=True, inplace=True)
froude_df.set_index('time', drop=True, inplace=True)

_wind_lim = (225, 340)

_odf = criteria_df.merge(precip_df, on = 'time').merge(wind_df,on='time').merge(froude_df,on='time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

df = collect_seasons()

df['Region']='Lemhi East Lower'
df['Precip']='Schwartz Lake'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,393.803849,27.587795
std,79.293617,11.753627
min,213.289497,12.429379
25%,346.525031,18.752558
50%,376.262165,24.125871
75%,448.401615,32.938591
max,583.950139,60.911988


In [ ]:
print(df.describe().to_markdown())

## Gunsight Peak Precip with Lemhi East Upper Criteria

| Precip   | Criteria         | Wind    | Generator  | Froude Var |
| -------- | ---------------- | ------- | ---------- | ---------- |
| Gunsight | Lemhi East Upper | 225-315 | A          | Fr_LH      |



In [48]:
#Gunsight Peak Precip with Lemhi East Upper Criteria
precip_df = pd.DataFrame({'time':Gunsight_Time, 'precip' :Gunsight_Precip})
criteria_df = pd.DataFrame({'time':LEUpper_Time, 'temp':LEUpper_GS_Temp,
                             'lwc':LEUpper_GS_LWC})
wind_df = pd.DataFrame({'time':A_Time, 'wind':A_windDir})  
froude_df = pd.DataFrame({'time':Fr_A_Time, 'Fr':LH_A})
precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)
wind_df.set_index('time', drop=True, inplace=True)
froude_df.set_index('time', drop=True, inplace=True)


_odf = criteria_df.merge(precip_df, on = 'time').merge(wind_df,on='time').merge(froude_df,on='time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

_wind_lim = (225, 315)

df = collect_seasons()

df['Region']='Lemhi East Upper'
df['Precip']='Gunsight'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,248.393664,18.316535
std,62.553329,8.199426
min,113.938819,6.392184
25%,212.383613,12.728835
50%,243.297851,16.651076
75%,275.729360,22.114649
max,466.630831,37.291769


In [ ]:
print(df.describe().to_markdown())

## Moonshine Precip with Pahsimeroi North Criteria

| Precip    | Criteria         | Wind    | Generator  | Froude Var |
| --------- | ---------------- | ------- | ---------- | ---------- |
| Moonshine | Pahsimeroi North | 250-340 | I          | Fr_LH      |


In [49]:
#Moonshine Precip with Pahsimeroi North Criteria
precip_df = pd.DataFrame({'time':Moonshine_Time, 'precip' :Moonshine_Precip})
criteria_df = pd.DataFrame({'time':PNorth_Time, 'temp':PNorth_GS_Temp,
                             'lwc':PNorth_GS_LWC})
wind_df = pd.DataFrame({'time':I_Time, 'wind':I_windDir})  
froude_df = pd.DataFrame({'time':Fr_I_Time, 'Fr':LH_I})
precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)
wind_df.set_index('time', drop=True, inplace=True)
froude_df.set_index('time', drop=True, inplace=True)


_odf = criteria_df.merge(precip_df, on = 'time').merge(wind_df,on='time').merge(froude_df,on='time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

_wind_lim = (205, 340)

df = collect_seasons()

df['Region']='Pahsimeroi North'
df['Precip']='Moonshine'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,370.644433,64.676615
std,86.221006,23.872457
min,187.994788,26.832728
25%,316.463895,45.117242
50%,368.375461,59.132516
75%,410.326606,81.631879
max,566.318840,120.954444


In [ ]:
print(df.describe().to_markdown())

## Gunsight Peak Precip with Lemhi East Upper Criteria

| Precip   | Criteria         | Wind    | Generator  | Froude Var |
| -------- | ---------------- | ------- | ---------- | ---------- |
| Gunsight | Lemhi East Upper | 180-300 | B          | Fr_LH      |



In [50]:
#Gunsight Peak Precip with Lemhi East Upper Criteria
precip_df = pd.DataFrame({'time':Gunsight_Time, 'precip' :Gunsight_Precip})
criteria_df = pd.DataFrame({'time':LEUpper_Time, 'temp':LEUpper_GS_Temp,
                             'lwc':LEUpper_GS_LWC})
wind_df = pd.DataFrame({'time':B_Time, 'wind':B_windDir})  
froude_df = pd.DataFrame({'time':Fr_B_Time, 'Fr':LH_B})
precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)
wind_df.set_index('time', drop=True, inplace=True)
froude_df.set_index('time', drop=True, inplace=True)


_odf = criteria_df.merge(precip_df, on = 'time').merge(wind_df,on='time').merge(froude_df,on='time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

_wind_lim = (180, 300)

df = collect_seasons()

df['Region']='Lemhi East Upper'
df['Precip']='Gunsight Peak'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()



,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,248.393664,19.553826
std,62.553329,9.880367
min,113.938819,7.504033
25%,212.383613,12.429923
50%,243.297851,19.282478
75%,275.729360,22.469830
max,466.630831,51.145077


In [ ]:
print(df.describe().to_markdown())

## Schwartz Precip with Lemhi East Lower Criteria

| Precip   | Criteria         | Wind    | Generator  | Froude Var |
| -------- | ---------------- | ------- | ---------- | ---------- |
| Schwartz | Lemhi East Lower | 180-300 | D          | Fr_LH      |



In [51]:
#Schwartz Precip with Lemhi East Lower Criteria
precip_df = pd.DataFrame({'time':Schwartz_Time, 'precip' :Schwartz_Precip})
criteria_df = pd.DataFrame({'time':LELower_Time, 'temp':LELower_GS_Temp,
                             'lwc':LELower_GS_LWC})
wind_df = pd.DataFrame({'time':D_Time, 'wind':D_windDir})  
froude_df = pd.DataFrame({'time':Fr_D_Time, 'Fr':LH_D})
precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)
wind_df.set_index('time', drop=True, inplace=True)
froude_df.set_index('time', drop=True, inplace=True)


_odf = criteria_df.merge(precip_df, on = 'time').merge(wind_df,on='time').merge(froude_df,on='time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

_wind_lim = (180, 300)

df = collect_seasons()

df['Region']='Lemhi East Lower'
df['Precip']='Schwartz'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,393.803849,30.243065
std,79.293617,10.833139
min,213.289497,13.274321
25%,346.525031,23.299529
50%,376.262165,27.449556
75%,448.401615,37.089642
max,583.950139,55.384579


In [ ]:
print(df.describe().to_markdown())